# Nss Midcourse project: Creating pathfinder 2e monster encounter


notes about base builds

abilty scores
* All characters start with a 10 in stats
* ability ancester flaws = drops to 8
* ancester ability boosts = rises to 12

Misc/other categories
* column for 'situational bonus'-> things like darkvision
* Maybe have a bonus for speed? Like elves get a bonus because more than base speed, but dwarfs get a minus because less than base speed
* Some ancestries have equipment (like dwarf clan dagger)


In [1]:
#Import libraries 
import pandas as pd
import numpy as np
import requests
import matplotlib as plt


import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

## Try loading data from archives of nethys website

### Steps of the code:
* Pull out the 

### Pull out all ancestry information 
* dwarf= 'https://2e.aonprd.com/Ancestries.aspx?ID=59'
* elf= https://2e.aonprd.com/Ancestries.aspx?ID=60
* halfling= https://2e.aonprd.com/Ancestries.aspx?ID=63
* human= https://2e.aonprd.com/Ancestries.aspx?ID=64
* orc= https://2e.aonprd.com/Ancestries.aspx?ID=66

In [183]:
input_ancestry_name= ['dwarf', 'elf', 'halfling', 'human', 'orc']
input_ancestry_number= [59, 60, 63, 64, 66]

#create a list to pull out the attributes most important
information_flags= ['Hit Points', 'Speed', 'Attribute Boosts', 'Attribute Flaw'] #, 'Darkvision', 'Low-Light Vision']



### Input functions for webscrapping 

In [215]:
def webscrape_2groups(group_1, id_number): 
    URL = f'https://2e.aonprd.com/{group_1}.aspx?ID={id_number}'
    
    headers = {
        "User-Agent": "MyPythonScraper"
    }
    
    #Pull webpage information
    response = requests.get(URL, headers=headers)
    response.status_code
    
    #stop code if the webpage is bad
    assert response.status_code == 200
    
    #convert webpage into a soup object
    soup = BeautifulSoup(response.text)

    return soup

def ancestry_webscrape_to_dataframe(input_soup, section_filter): 
    soup= input_soup

    ######
    ## Pull out ancestry information from a webpage section 
    #####
    ### insepcting the datapage- all ancenstries are within a <h2 class> order
    ancestry_title= soup.findAll('h2', attrs={"class":"title"})
    
    ancestry_page_subtitles=[] #A list of just the header subtitles
    for section_label in ancestry_title:
        output_text= section_label.text
        ancestry_page_subtitles.append(output_text)

    ####
    #Webscrape just the section of the webpage with the ancestry stats
    ####
    #Webscrape just the section of the webpage with the ancestry stats
    for section in range(0, len(soup.findAll('h1'))):
        #loop through all the h1 class items to find the index for '{ancestry} Mechanics' section
        current_h1_item= soup.findAll('h1')[section]
        if current_h1_item.text.__contains__(section_filter):
            #Save the NEXT h1 section text so the loop will only pull out the information in the mechanics section
            end_line_flag= soup.findAll('h1')[section+1].text
            line_value= current_h1_item.next_sibling
            line_text= line_value.text
            stop = False
            output_by_line=[]
            #Create a while loop to pull out the informaiton and save the text info in the output list, but stop once it hits the next h1 section
            while line_text != end_line_flag and not stop:
                output_by_line.append(line_text)
                
                line_value= line_value.next_sibling

                #conditional makes sure that the line_value has a next_sibling object, otherwise it ends the 'while' loop
                if line_value:
                    line_text= line_value.text
                else:
                    stop = True

    ################
    # convert webscraped text to dataframe and drop empty
    
    #convert webscrapped information to a dataframe for easier cleaning 
    output_df= pd.DataFrame(output_by_line, columns=['output'])
    #remove any rows that are blank (from being HTML code)
    output_df= output_df.replace('', 'nan').replace(',', 'nan')
    output_df= output_df[output_df['output'] != 'nan']
    #reset dataframe index to make using .iloc easiear
    output_df= output_df.reset_index(drop=True)

    return output_df, ancestry_page_subtitles

def pull_attribute_info_from_archives(input_df, information_flags, ancestry_page_subtitles, ancestry_name):
    output_df= input_df

    #make lists to store the filtered information to match up later
    index_value_attributes=[] #save index as output list for future filtering
    name_value_attributes=[] #save name of attribute to check filter
    body_text_saved= []
    
    #First loop through each of the page subtitles and see if it is one we want to pull information from 
    for index, subtitle in enumerate(ancestry_page_subtitles):
    
        #If it IS one we care about, add the information between the flags to the save list
        if subtitle in information_flags:
            #save the attribute name to the attribute name list 
            name_value_attributes.append(subtitle)
            
            #print("flag", subtitle)
            #pull out the column from the dataframe 
            attribute_cell= output_df[output_df['output'] == subtitle]
            #determine the index for futher filtering 
            attribute_index= int(attribute_cell.index.values)
            #print("attribute index= ", attribute_index)
    
            #identify the NEXT attribute subtitle, and its index in the dataframe 
            next_attribute_name= ancestry_page_subtitles[index + 1]
            next_attribute_cell= output_df[output_df['output'] == next_attribute_name]
            next_attribute_index= int(next_attribute_cell.index.values)
            #print("next attribute index= ", next_attribute_index)
    
            #Now pull out all the text BETWEEN these two attribute titles into one list
            save_text_for_list=[]
            #using a for loop will help comdense any subtitle sections with mutliple lines of text into just one list to save
            for body_text in range (attribute_index+1, next_attribute_index):
                text_to_save= output_df.loc[body_text].values[0]
                #print(body_text)
                #print("text pulled out: ", text_to_save)
                save_text_for_list.append(text_to_save)
            body_text_saved.append(save_text_for_list) #this will save multiple lined items as one list
    
    # Save all this information into a final dataframe to clean up formatting 
    clean_webpage_dict= {'attribute name':name_value_attributes, 
                         'attribute text': body_text_saved}
    
    ancestry_webpage_data= pd.DataFrame(clean_webpage_dict)
    ancestry_webpage_data.columns=['Attribute', f'{ancestry_name}']

    return ancestry_webpage_data

### Run webscraping functions for all ancestries, then combine the final output files into one dataframe

In [216]:

#create final dataframe to save all ancestry info
ancestry_data= pd.DataFrame(information_flags, columns=['Attribute'])

#loop through all the ancestries of interest, pull out the relevant data, and add it to the main output dataframe
for index, name in enumerate(input_ancestry_name):
    #pull out the filter values from the two info lists 
    ancestry_name= name
    ancestry_number= input_ancestry_number[index]
    #pull up webpage using the standard url layout 
    ancestry_soup= webscrape_2groups('Ancestries', 
                                     f'{ancestry_number}')
    ancestry_webscrape, ancestry_subtitles= ancestry_webscrape_to_dataframe(ancestry_soup, 
                                                                            'Mechanics')
    ancestry_output_df = pull_attribute_info_from_archives(ancestry_webscrape, 
                                                           information_flags, 
                                                           ancestry_subtitles, 
                                                           f'{ancestry_name}')
    ancestry_data= pd.merge(ancestry_data, 
                            ancestry_output_df, 
                            on='Attribute', 
                            how='left')
ancestry_data

#save ancestry output as a csv file 

ancestry_data.to_csv('./Data/ancestry_attribute.csv')

#view final dataframe 
ancestry_data

C:\Users\Mullo\AppData\Local\Temp\ipykernel_24296\1166462714.py:93: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  attribute_index= int(attribute_cell.index.values)
C:\Users\Mullo\AppData\Local\Temp\ipykernel_24296\1166462714.py:99: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_attribute_index= int(next_attribute_cell.index.values)
C:\Users\Mullo\AppData\Local\Temp\ipykernel_24296\1166462714.py:93: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  attribute_index= 

,Attribute,dwarf,elf,halfling,human,orc
0,Hit Points,[10],[6],[6],[8],[10]
1,Speed,[20 feet],[30 feet],[25 feet],[25 feet],[25 feet]
2,Attribute Boosts,"[Constitution, Wisdom, Free]","[Dexterity, Intelligence, Free]","[Dexterity, Wisdom, Free]",[Two free ability boosts],[Two free ability boosts]
3,Attribute Flaw,[Charisma],[Constitution],[Strength],NaN,NaN


----

### Webscrape for all classes 

* fighter = https://2e.aonprd.com/Classes.aspx?ID=35
* sorcerer = https://2e.aonprd.com/Classes.aspx?ID=62
* rogue= https://2e.aonprd.com/Classes.aspx?ID=37
* barbarian= https://2e.aonprd.com/Classes.aspx?ID=57
* champion= https://2e.aonprd.com/Classes.aspx?ID=58

In [236]:
class_choices= ['fighter', 'sorcerer', 'rogue', 'barbarian', 'champion']
class_webpage_id= [35, 62, 37, 57, 58]

class_proficiencies= ['Perception',
 'Saving Throws',
 'Skills',
 'Attacks',
 'Defenses',
 'Class DC',
 'Ancestry and Background',
                      'Attribute Boosts']

In [237]:
def pull_text_between_sections_multiple(soup, first_section, second_section, section_sublist, filter_title, group_title):
    class_title= soup.findAll(first_section, attrs={"class":"title"})
    
    class_page_subtitles=[] #A list of just the header subtitles
    for section_label in class_title:
        output_text= section_label.text
        class_page_subtitles.append(output_text)
    
    output_section_list=[]
    for data_name in section_sublist:
        section_filter= data_name
        #Webscrape just the section of the webpage with the ancestry stats
        for section in range(0, len(soup.findAll(first_section))):
            #loop through all the h1 class items to find the index for '{ancestry} Mechanics' section
            current_h1_item= soup.findAll(first_section)[section]
            if current_h1_item.text.__contains__(section_filter):
                #Save the NEXT h1 section text so the loop will only pull out the information in the mechanics section
                end_line_flag= soup.findAll(first_section)[section+1].text
                line_value= current_h1_item.next_sibling
                line_text= line_value.text
                stop = False
                output_by_line=[]
                #Create a while loop to pull out the informaiton and save the text info in the output list, but stop once it hits the next h1 section
                while line_text != end_line_flag and not stop:
                    output_by_line.append(line_text)
                    
                    line_value= line_value.next_sibling
        
                    #conditional makes sure that the line_value has a next_sibling object, otherwise it ends the 'while' loop
                    if line_value:
                        line_text= line_value.text
                    else:
                        stop = True
        
        ################
        # convert webscraped text to dataframe and drop empty
        
        #convert webscrapped information to a dataframe for easier cleaning 
        output_df= pd.DataFrame(output_by_line, columns=['output'])
        #remove any rows that are blank (from being HTML code)
        output_df= output_df.replace('', 'nan').replace(',', 'nan')
        output_df= output_df[output_df['output'] != 'nan']
        #reset dataframe index to make using .iloc easiear
        output_df= output_df.reset_index(drop=True)
        
        output_as_list= output_df['output'].to_list()
        output_section_list.append(output_as_list) 
    
    #Convert it into a dataframe 
    output_dict= {f'{filter_title}' : section_sublist, 
                  f'{group_title}': output_section_list}
    class_df= pd.DataFrame(output_dict)
    class_df

    return class_df, class_page_subtitles

In [238]:
#create final dataframe to save all ancestry info
class_proficency_data= pd.DataFrame(class_proficiencies, 
                                    columns=['initial proficencies'])
class_proficency_data

,initial proficencies
0,Perception
1,Saving Throws
2,Skills
3,Attacks
4,Defenses
5,Class DC
6,Ancestry and Background
7,Attribute Boosts


In [241]:
#create final dataframe to save all ancestry info
class_proficency_data= pd.DataFrame(class_proficiencies, 
                                    columns=['initial proficencies'])

#loop through all the ancestries of interest, pull out the relevant data, and add it to the main output dataframe
for index, name in enumerate(class_choices):
    #pull out the filter values from the two info lists 
    class_name= name
    class_number= class_webpage_id[index]
    #pull up webpage using the standard url layout 
    class_soup= webscrape_2groups('Classes', 
                                  f'{class_number}')
    
    class_output_df, class_page_subtitles = pull_text_between_sections_multiple(class_soup, 
                                                             'h2', 
                                                             'h1', 
                                                             class_proficiencies, 
                                                             'initial proficencies', 
                                                             f'{name}')
    class_proficency_data= pd.merge(class_proficency_data, class_output_df, on='initial proficencies', how='left')


#save ancestry output as a csv file 

class_proficency_data.to_csv('./Data/class_initial_proficiencies.csv')

#view final dataframe 
class_proficency_data


,initial proficencies,fighter,sorcerer,rogue,barbarian,champion
0,Perception,[Expert in Perception],[You remain alert to threats around you. Your ...,[You notice things almost impossible for an or...,[Your instinct heightens each of your senses f...,[You remain alert to threats around you. Your ...
1,Saving Throws,"[Expert in Fortitude, Expert in Reflex, Traine...","[Trained in Fortitude, Trained in Reflex, Expe...","[Trained in Fortitude, Expert in Reflex, Exper...","[Expert in Fortitude, Trained in Reflex, Exper...","[Expert in Fortitude, Trained in Reflex, Exper..."
2,Skills,"[Trained in your choice of , Acrobatics, or ,...",[Trained in two skills determined by your bloo...,"[Trained in , Stealth, Trained in one or more...","[Trained in , Athletics, Trained in a number ...","[Trained in , Religion, Trained in one skill ..."
3,Attacks,"[Expert in simple weapons, Expert in martial ...","[Trained in simple weapons, Trained in unarme...","[Trained in simple weapons, Trained in martia...","[Trained in simple weapons, Trained in martia...","[Trained in simple weapons, Trained in martia..."
4,Defenses,"[Trained in all armor, Trained in unarmored d...","[Untrained in all armor, Trained in unarmored...","[Trained in light armor, Trained in unarmored...","[Trained in light armor, Trained in medium ar...","[Trained in all armor, Trained in unarmored d..."
5,Class DC,"[Trained in fighter class DC, Class Features, ...",[Trained in sorcerer class DC],"[Trained in rogue class DC, Class Features, Yo...","[Trained in barbarian class DC, Class Features...",[Trained in champion class DC]
6,Ancestry and Background,[In addition to what you get from your class a...,[In addition to the abilities provided by your...,[In addition to what you get from your class a...,[In addition to the abilities provided by your...,[In addition to the abilities provided by your...
7,Attribute Boosts,[In addition to what you get from your class a...,[In addition to what you get from your class a...,[In addition to what you get from your class a...,[In addition to what you get from your class a...,[In addition to what you get from your class a...


----

**Current stopping point**: next task list
* Create single streamlined functions for webscraping, since all have similar process (condense repeated code)
* webscrape final .csv for the ancestry and class feats
* Make a dataframe for each level skill/feat slot to be filled in based on build (aka character build blueprint)
* clean up text from webscrapping-> remove flavor text and only have most relevant information

----

## pull information from rpgbot 
### This will create a term list of feats and equipment most reccomended to be useful for a build in a given ancestry. This will be used for any necessary optimization build decisions once full stats are created.

In [ ]:
ancestry_name= 'dwarf'


URL = f"https://rpgbot.net/p2/characters/ancestries/{ancestry_name}/"

headers = {
    "User-Agent": "MyPythonScraper"
}

response = requests.get(URL, headers=headers)
response.status_code
assert response.status_code == 200

#convert it into a soup object
soup = BeautifulSoup(response.text)

#pull out a list of all the best feats to chose for the specific ancestry 
green_rating_repbot= soup.findAll('span', attrs={"class":"rating-green"})

output_green_choice= []
for item in green_rating_repbot:
    output_text = item.get_text()
    output_green_choice.append(output_text)

blue_rating_repbot= soup.findAll('span', attrs={"class":"rating-blue"})

output_blue_choice= []
for item in blue_rating_repbot:
    output_text = item.get_text()
    output_blue_choice.append(output_text)

#combine both lists to have a filter option 

best_choice_list= output_green_choice + output_blue_choice
best_choice_list